In [6]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import sympy as sp

In [49]:
import pandas as pd

file='/content/Skyserver_SQL11_1_2023 4_20_18 PM.csv'

cat = pd.read_csv(file,delimiter=',',header=1)

row_ind = [8,111,72,80,25]
cat_sel = cat.iloc[row_ind]

js2=cat_sel['SII_6717_flux']/cat_sel['SII_6731_flux']
jo2=cat_sel['OII_3729_flux']/cat_sel['OII_3726_flux']
jo3=(cat_sel['OIII_5007_flux']+cat_sel['OIII_4959_flux'])/cat_sel['OIII_4363_flux']

js2=np.array(js2)
jo3=np.array(jo2)

In [50]:
jo3

array([0.62815423, 0.96015824, 0.60791684, 1.36192986, 1.16688619])

In [3]:
cat_sel

,plate,fiberid,mjd,z,zwarning,class,specObjID,h_beta_flux,h_beta_flux_err,h_alpha_flux,...,NII_6584_flux,NII_6584_flux_err,SII_6717_flux,SII_6717_flux_err,SII_6731_flux,SII_6731_flux_err,OII_3726_flux,OII_3726_flux_err,OII_3729_flux,OII_3729_flux_err
8,744,184,52251,0.039813,0,GALAXY,837720145991329792,45.00267,4.923592,451.9799,...,338.92880,8.508451,143.42070,7.395702,105.78010,7.220693,73.89171,13.548730,46.41539,13.429430
111,2274,364,53726,0.091606,0,GALAXY,2560396506230188032,44.51060,2.325790,479.9265,...,317.53760,3.312216,89.50102,2.775858,74.13079,2.606775,36.12391,4.447652,34.68467,4.438820
72,1875,323,54453,0.038607,0,GALAXY,2111151185602832384,20.80641,4.085865,209.9367,...,113.72500,3.949086,56.20755,3.596226,41.56231,3.307410,40.67757,13.023540,24.72858,13.261360
80,2795,589,54563,0.256993,0,GALAXY,3147052219266787328,11.15065,1.703433,120.4203,...,59.37738,3.275947,25.92093,2.460145,26.61004,3.066978,10.82649,2.381013,14.74492,2.400014
25,939,225,52636,0.027827,0,GALAXY,1057281904279054336,57.87104,2.881205,618.1323,...,626.03000,6.251127,121.98640,4.250310,115.17290,4.192435,66.09684,8.244592,77.12749,8.031549


In [4]:
#SII
a6716=2.6e-4
a6731=8.8e-4

d32_d52=7.47
s32_d52=4.14
s32_d32=2.76
d52_d32=7.47

g52=6
g32=4

T=10000
C=8.62e-6/np.sqrt(T)

In [21]:
#OIII

c=299792458 #m/s
h=6.626070e-34 #J/S
k=1.380649e-23 #J/K

adp1_o3=2e-2
adp2_o3=6.8e-3
asd_o3=1.6
asp_o3=2.3e-1

omg_ds_o3=0.58
omg_ps_o3=0.29
omg_pd_o3=2.29

gamma_o3=(omg_ds_o3+omg_ps_o3)/omg_ps_o3

gs=1
gd=5

lsd_4363=4363.2e-10
e_sd_o3=(h*c)/lsd_4363
nu_sd_o3=(c)/lsd_4363


ldp1_5007=5006.9e-10
e_dp1_o3=c/ldp1_5007

ldp2_4959=4958.9e-10
e_dp2_o3=c/ldp2_4959

In [55]:
from sympy.core import S
#Ne para SII
x1 = sp.symbols('x')
ecuacion1 = (((a6716*g52)*(1+(d32_d52/s32_d52)+(d32_d52/s32_d32)+((a6731*g32)/(C*x*s32_d32))))/((a6731*g32)*(1+(d52_d32/s32_d32)+(d52_d32/s32_d52)+((a6716*g52)/(C*x*s32_d52)))))-js2[0]
ne1 = sp.solve(ecuacion1, x1)
ne1 = np.array(ne1)

In [ ]:
#T con ese ne_1 para OIII
x2 = sp.symbols('x2')
ecuacion2 = ((((adp1_o3*e_dp1_o3)+(adp2_o3*e_dp2_o3))*(1+((asd_o3+asp_o3)*gs/(gamma_o3*(8.62e-6/sp.sqrt(x2))*ne1[0]*omg_ps_o3))+(omg_ds_o3*(sp.exp(-(e_sd_o3/(k*x2))))/(gamma_o3*omg_pd_o3))+(asd_o3*gs*(sp.exp(-(e_sd_o3/(k*x2))))/(gamma_o3*(8.62e-6/sp.sqrt(x2))*ne1[0]*omg_pd_o3))))/((asd_o3*nu_sd_o3*gs*(sp.exp(-(e_sd_o3/(k*x2))))/gd)*(1+(omg_ds_o3*(sp.exp(-(e_sd_o3/(k*x2))))/(gamma_o3*omg_pd_o3))+((adp1_o3+adp2_o3)*gd/(gamma_o3*(8.62e-6/sp.sqrt(x2))*ne1[0]*omg_pd_o3)))))-jo3[0]
T2 = sp.solve(ecuacion2, x2)

In [10]:
solucion

[39.6223083304658]

In [12]:
n_1=(((a6716*g52*a6731*g32)/(C*s32_d32))-((a6716*g52*a6731*g32)/(C*s32_d52)))
d_1=(js2[0])*(1+(d52_d32/s32_d32)+(d52_d32/s32_d52))*((a6731*g32)-(a6716*g52))

ne_1=n_1/d_1

In [13]:
ne_1

166.1290537050042

In [ ]:
def valores(js2,jo2):

  n_1=(((a6716*g52*a6731*g32)/(C*s32_d32))-((a6716*g52*a6731*g32)/(C*s32_d52)))
  d_1=(js2)*(1+(d52_d32/s32_d32)+(d52_d32/s32_d52))*((a6731*g32)-(a6716*g52))

  ne_1=n_1/d_1

  n_2=8.62e-6*(jo2)*(1+(o_d52_d32/o_s32_d32)+(o_d52_d32/o_s32_d52))*((a3726*g32)-(a3729*g52))
  d_2=(((a3729*g52*a3726*g32)/(ne_1*o_s32_d32))-((a3726*g52*a3729*g32)/(ne_1*o_s32_d52)))

  T_2=np.power((n_2/d_2),2)

  print('Ne 1:',ne_1)
  print('T 1:',T_2)

  C_3=8.62e-6/np.sqrt(T_2)
  n_3=(((a6716*g52*a6731*g32)/(C_3*s32_d32))-((a6716*g52*a6731*g32)/(C_3*s32_d52)))
  d_3=(js2)*(1+(d52_d32/s32_d32)+(d52_d32/s32_d52))*((a6731*g32)-(a6716*g52))

  ne_3=n_3/d_3

  n_4=8.62e-6*(jo2)*(1+(o_d52_d32/o_s32_d32)+(o_d52_d32/o_s32_d52))*((a3726*g32)-(a3729*g52))
  d_4=(((a3729*g52*a3726*g32)/(ne_3*o_s32_d32))-((a3726*g52*a3729*g32)/(ne_3*o_s32_d52)))

  T_4=np.power((n_4/d_4),2)

  print('Ne 2:',ne_3)
  print('T 2:',T_4)

  C_5=8.62e-6/np.sqrt(T_4)
  n_5=(((a6716*g52*a6731*g32)/(C_5*s32_d32))-((a6716*g52*a6731*g32)/(C_5*s32_d52)))
  d_5=(js2)*(1+(d52_d32/s32_d32)+(d52_d32/s32_d52))*((a6731*g32)-(a6716*g52))

  ne_5=n_5/d_5

  n_6=8.62e-6*(jo2)*(1+(o_d52_d32/o_s32_d32)+(o_d52_d32/o_s32_d52))*((a3726*g32)-(a3729*g52))
  d_6=(((a3729*g52*a3726*g32)/(ne_5*o_s32_d32))-((a3726*g52*a3729*g32)/(ne_5*o_s32_d52)))

  T_6=np.power((n_6/d_6),2)

  print('Ne 3:',ne_5)
  print('T 3:',T_6)

  C_7=8.62e-6/np.sqrt(T_6)
  n_7=(((a6716*g52*a6731*g32)/(C_7*s32_d32))-((a6716*g52*a6731*g32)/(C_7*s32_d52)))
  d_7=(js2)*(1+(d52_d32/s32_d32)+(d52_d32/s32_d52))*((a6731*g32)-(a6716*g52))

  ne_7=n_7/d_7

  n_8=8.62e-6*(jo2)*(1+(o_d52_d32/o_s32_d32)+(o_d52_d32/o_s32_d52))*((a3726*g32)-(a3729*g52))
  d_8=(((a3729*g52*a3726*g32)/(ne_7*o_s32_d32))-((a3726*g52*a3729*g32)/(ne_7*o_s32_d52)))

  T_8=np.power((n_8/d_8),2)

  print('Ne 4:',ne_7)
  print('T 4:',T_8)

In [ ]:
valores(js2[0],jo2[0])

Ne 1: 166.1290537050042
T 1: 423.39654403482814
Ne 2: 108.09839497146872
T 2: 179.26463350063605
Ne 3: 70.33846720247512
T 3: 75.90002629183944
Ne 4: 45.7684868466317
T 4: 32.13580882411742


In [ ]:
valores(js2[1],jo2[1])

Ne 1: 186.56239115990337
T 1: 1247.5483381395852
Ne 2: 208.37844385636913
T 2: 1556.3768559948414
Ne 3: 232.74560105088477
T 3: 1941.6553602152749
Ne 4: 259.96218133712597
T 4: 2422.3089178763803


In [ ]:
valores(js2[2],jo2[2])

Ne 1: 166.55530791833564
T 1: 398.5922612976061
Ne 2: 105.15330057910977
T 2: 158.8757907663393
Ne 3: 66.38765681428855
T 3: 63.3266607070006
Ne 4: 41.9132918607343
T 4: 25.241516891629644


In [ ]:
valores(js2[3],jo2[3])

Ne 1: 231.2322494242783
T 1: 3855.9362368428892
Ne 2: 454.0600847103979
T 2: 14868.244262598095
Ne 3: 891.616809681768
T 3: 57331.001830383444
Ne 4: 1750.826734338776
T 4: 221064.6874522815


In [ ]:
valores(js2[4],jo2[4])

Ne 1: 212.66319942721526
T 1: 2394.2265457075064
Ne 2: 329.0599021347584
T 2: 5732.320752170503
Ne 3: 509.16387736372826
T 3: 13724.474513356618
Ne 4: 787.8439528189523
T 4: 32859.50120576453
